# Pauli Propagation of Parametrized Circuits: Tutorial

* **Step 0:** Import the necessary libraries

In [1]:
import pennylane as qml # To define the circuit
from pprop.propagator import Propagator # For Pauli Propagation

/home/samonaco/Pauli-Propagator/.venv/lib/python3.12/site-packages/pennylane/operation.py:2622: PennyLaneDeprecationWarning: Observable is deprecated and will be removed in v0.43. A generic Operator class should be used instead. If defining an Operator, set the is_hermitian property to True. If checking if an Operator is Hermitian, check the is_hermitian property. 
  warnings.warn(


In [4]:
# Function of parameters

def ansatz(params : list[float]):
    for i in range(3):
        qml.H(wires = i)

    for i in range(2):
        qml.CNOT(wires = [i, i+1])

    qml.RX(params[0], wires = 0)
    qml.CRZ(params[1], wires = [1,2])
    qml.RZ(params[2], wires = 2)

    for i in range(2):
        qml.CNOT(wires = [i+1, i])
        qml.CY(wires = [i+1, i])
    
    qml.CRY(params[3], wires = [0,1])
    qml.CRX(params[4], wires = [2,0])
   
    qml.RZ(params[5], wires = 0)

    for i in range(2):
        qml.CZ(wires = [i+1, i])
        qml.CNOT(wires = [i+1, i])

    for i in range(3):
        qml.Hadamard(wires = i)
        
    for i in range(3):
        qml.T(wires = i)
        
    for i in range(3):
        qml.S(wires = i)

    for i in range(2):
        qml.CZ(wires = [i, i+1])
    
    return [qml.expval(qml.PauliZ(qubit)) for qubit in range(1)] + [qml.expval(qml.PauliX(0)@qml.PauliX(1)@qml.PauliX(2))] + [qml.expval(qml.PauliY(2))] + [qml.expval(-qml.PauliX(0)@qml.PauliX(1)@qml.PauliX(2) + 13*qml.PauliZ(2))]

In [9]:
# Function of parameters

def pansatz(params : list[float]):
    for i in range(3):
        qml.H(wires = i)

    for i in range(2):
        qml.CNOT(wires = [i, i+1])

    qml.RX(params[0], wires = 0)
    qml.CRZ(2*params[1], wires = [1,2])
    qml.RZ(params[2], wires = 2)

    for i in range(2):
        qml.CNOT(wires = [i+1, i])
        qml.CY(wires = [i+1, i])
    
    qml.CRY(2*params[3], wires = [0,1])
    qml.CRX(2*params[4], wires = [2,0])
   
    qml.RZ(params[5], wires = 0)

    for i in range(2):
        qml.CZ(wires = [i+1, i])
        qml.CNOT(wires = [i+1, i])

    for i in range(3):
        qml.Hadamard(wires = i)
        
    for i in range(3):
        qml.T(wires = i)
        
    for i in range(3):
        qml.S(wires = i)

    for i in range(2):
        qml.CZ(wires = [i, i+1])
    
    return [qml.expval(qml.PauliZ(qubit)) for qubit in range(1)] + [qml.expval(qml.PauliX(0)@qml.PauliX(1)@qml.PauliX(2))] + [qml.expval(qml.PauliY(2))] + [qml.expval(-qml.PauliX(0)@qml.PauliX(1)@qml.PauliX(2) + 13*qml.PauliZ(2))]

In [10]:
prop = Propagator(
    ansatz, 
    k1 = None, # Cutoff on the Pauli Weight
    k2 = None, # Cutoff on the frequencies
)
prop.show()
prop.propagate(debug=False)

0: ──H─╭●──RX(0.00)───────────╭X────────╭Y───────╭●────────╭RX(4.00)──RZ(5.00)─╭Z─╭X──H──T──S─── ···
1: ──H─╰X─╭●────────╭●────────╰●────────╰●─╭X─╭Y─╰RY(3.00)─│───────────────────╰●─╰●─╭Z─╭X──H──T ···
2: ──H────╰X────────╰RZ(1.00)──RZ(2.00)────╰●─╰●───────────╰●────────────────────────╰●─╰●──H──T ···

0: ··· ────╭●────┤  <Z> ╭<X@X@X>      ╭<𝓗(-1.00,13.00)>
1: ··· ──S─╰Z─╭●─┤      ├<X@X@X>      ├<𝓗(-1.00,13.00)>
2: ··· ──S────╰Z─┤      ╰<X@X@X>  <Y> ╰<𝓗(-1.00,13.00)>
Propagating (1.0000)*Z0
Propagating (1.0000)*X0 X1 X2
Propagating (1.0000)*Y2
Propagating (-1.0000)*X0 X1 X2 + (13.0000)*Z2


In [11]:
random_params = qml.numpy.random.uniform(-qml.numpy.pi, qml.numpy.pi, prop.num_params)

In [12]:
prop_output = prop(random_params)
prop_output

array([0.11477842, 0.43733517, 0.04724619, 3.56530458])

In [13]:
device = qml.device('default.qubit', wires = 3)

circuit = qml.QNode(pansatz, device)
pennylane_output = circuit(random_params)
pennylane_output

[tensor(0.11477842, requires_grad=True),
 tensor(0.43733517, requires_grad=True),
 tensor(0.04724619, requires_grad=True),
 tensor(3.56530458, requires_grad=True)]

How much is the difference between the two outputs?

In [14]:
params = qml.numpy.random.rand(prop.num_params)
print(prop(params))
print(circuit(params))  # via QNode

[ 0.82839135 -0.27935892 -0.27276472  5.64037829]
[tensor(0.82839135, requires_grad=True), tensor(-0.27935892, requires_grad=True), tensor(-0.27276472, requires_grad=True), tensor(5.64037829, requires_grad=True)]


In [15]:
for out1, out2 in zip(prop_output, pennylane_output):
    print(abs(out1 - out2), out1, out2)

1.3877787807814457e-17 0.11477842119290792 0.11477842119290793
6.661338147750939e-16 0.4373351714990865 0.43733517149908585
1.942890293094024e-16 0.047246190485643075 0.04724619048564288
5.773159728050814e-15 3.5653045840494557 3.56530458404945


You can inspect the explicit functions through `.expression()`:

In [ ]:
prop.exprs

[[(1.0, [], [np.int64(5), np.int64(3)]),
  (1.0,
   [np.int64(5), np.int64(4), np.int64(3), np.int64(1)],
   [np.int64(4), np.int64(3)]),
  (1.0, [np.int64(5), np.int64(4), np.int64(4), np.int64(3)], [np.int64(1)])],
 [(-0.3535533905932737,
   [np.int64(5), np.int64(4)],
   [np.int64(4), np.int64(1)]),
  (-0.3535533905932737,
   [np.int64(4), np.int64(3), np.int64(1)],
   [np.int64(5), np.int64(4), np.int64(3)]),
  (0.3535533905932737,
   [np.int64(5), np.int64(4), np.int64(4), np.int64(1)],
   [np.int64(3)]),
  (-0.3535533905932737,
   [np.int64(4), np.int64(4), np.int64(3)],
   [np.int64(5), np.int64(1)]),
  (0.3535533905932737, [np.int64(5)], [np.int64(3)]),
  (-0.3535533905932737,
   [np.int64(3)],
   [np.int64(4), np.int64(3), np.int64(2), np.int64(1)]),
  (-0.3535533905932737,
   [np.int64(4), np.int64(3), np.int64(3)],
   [np.int64(5), np.int64(2), np.int64(1)]),
  (0.3535533905932737,
   [np.int64(5), np.int64(4), np.int64(2)],
   [np.int64(3), np.int64(3), np.int64(1)]),
  (0.